### Use Cases:
    - Data Scientists need access to data in your data warehouse.  BigQuery makes this extremely easy.

### Description: 
    - BigQuery provides a magic keyword ```%%bigquery``` used for executing queries to view the results or to populate a dataframe.
    - You can easily save dataframes back to BigQuery.

### Reference:
    - https://cloud.google.com/bigquery/docs/visualize-jupyter

### Clean up / Reset script:
    n/a   


In [ ]:
from google.cloud import bigquery
import pandas
client = bigquery.Client()

### Write any SQL and execute in BigQuery

In [ ]:
%%bigquery
WITH MonthlyData AS
(
SELECT FORMAT_DATE("%B", taxi_trips.Pickup_DateTime) AS MonthName,
       FORMAT_DATE("%m", taxi_trips.Pickup_DateTime) AS MonthNumber,
       CASE WHEN taxi_trips.Payment_Type_Id = 1 THEN 'Credit'
            WHEN taxi_trips.Payment_Type_Id = 2 THEN 'Cash'
            WHEN taxi_trips.Payment_Type_Id = 3 THEN 'NoCharge'
            WHEN taxi_trips.Payment_Type_Id = 4 THEN 'Dispute'
         END AS PaymentDescription,
       SUM(taxi_trips.Total_Amount) AS Total_Amount
  FROM `taxi_dataset.taxi_trips` AS taxi_trips
 WHERE taxi_trips.Pickup_DateTime BETWEEN '2020-01-01' AND '2020-12-31' 
   AND Passenger_Count IS NOT NULL
   AND Payment_Type_Id IN (1,2,3,4)
 GROUP BY 1, 2, 3   
)
SELECT MonthName,
       FORMAT("%'d", CAST(Credit   AS INTEGER)) AS Credit,
       FORMAT("%'d", CAST(Cash     AS INTEGER)) AS Cash,
       FORMAT("%'d", CAST(NoCharge AS INTEGER)) AS NoCharge,
       FORMAT("%'d", CAST(Dispute  AS INTEGER)) AS Dispute
  FROM MonthlyData
 PIVOT(SUM(Total_Amount) FOR PaymentDescription IN ('Credit', 'Cash', 'NoCharge', 'Dispute'))
ORDER BY MonthNumber;

### Place your SQL Results directly into a Dataframe

In [ ]:
%%bigquery myDataframe
WITH MonthlyData AS
(
SELECT FORMAT_DATE("%B", taxi_trips.Pickup_DateTime) AS MonthName,
       FORMAT_DATE("%m", taxi_trips.Pickup_DateTime) AS MonthNumber,
       CASE WHEN taxi_trips.Payment_Type_Id = 1 THEN 'Credit'
            WHEN taxi_trips.Payment_Type_Id = 2 THEN 'Cash'
            WHEN taxi_trips.Payment_Type_Id = 3 THEN 'NoCharge'
            WHEN taxi_trips.Payment_Type_Id = 4 THEN 'Dispute'
         END AS PaymentDescription,
       SUM(taxi_trips.Total_Amount) AS Total_Amount
  FROM `taxi_dataset.taxi_trips` AS taxi_trips
 WHERE taxi_trips.Pickup_DateTime BETWEEN '2020-01-01' AND '2020-12-31' 
   AND Passenger_Count IS NOT NULL
   AND Payment_Type_Id IN (1,2,3,4)
 GROUP BY 1, 2, 3   
)
SELECT MonthName,
       CAST(Credit   AS INTEGER) AS Credit,
       CAST(Cash     AS INTEGER) AS Cash,
       CAST(NoCharge AS INTEGER) AS NoCharge,
       CAST(Dispute  AS INTEGER) AS Dispute
  FROM MonthlyData
 PIVOT(SUM(Total_Amount) FOR PaymentDescription IN ('Credit', 'Cash', 'NoCharge', 'Dispute'))
ORDER BY MonthNumber;

In [ ]:
# Show the results
display(myDataframe)

In [ ]:
# Loop through the dataframe
# You can now use BigQuery data just like any other datasource
for index, row in myDataframe.iterrows():
    print(("MonthName: {MonthName} | Credit: {Credit} | Cash {Cash}").format(MonthName=row['MonthName'].ljust(10),Credit=str(row['Credit']).ljust(10),Cash=str(row['Cash']).ljust(10)))

### Easy to query BigQuery with Adhoc SQL and place the results into a Dataframe

In [ ]:
sql="SELECT Payment_Type_Id, Payment_Type_Description FROM `taxi_dataset.payment_type`";

df_payment_type = client.query(sql).to_dataframe()

display(df_payment_type);


### Load data in your notebook into a BigQuery table

In [ ]:
# Load a table in BQ with results from your notebook

table_id = "taxi_dataset.notebook_monthly_data"

dataframe = pandas.DataFrame(
    myDataframe, # Your source data
    columns=[
        "MonthName",
        "Credit",
        "Cash",
        "NoCharge",
        "Dispute"
    ],
)

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("MonthName", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Credit", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("Cash", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("NoCharge", bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField("Dispute", bigquery.enums.SqlTypeNames.FLOAT64)
    ],
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(dataframe, table_id, job_config=job_config) 
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))

In [ ]:
%%bigquery
SELECT * FROM `taxi_dataset.notebook_monthly_data`;

In [ ]:
%%bigquery
DROP TABLE `taxi_dataset.notebook_monthly_data`;

### Show BigQuery results in a chart

In [ ]:
%matplotlib inline

In [ ]:
%%bigquery myDataframe
WITH MonthlyData AS
(
SELECT FORMAT_DATE("%B", taxi_trips.Pickup_DateTime) AS MonthName,
       FORMAT_DATE("%m", taxi_trips.Pickup_DateTime) AS MonthNumber,
       CASE WHEN taxi_trips.Payment_Type_Id = 1 THEN 'Credit'
            WHEN taxi_trips.Payment_Type_Id = 2 THEN 'Cash'
            ELSE 'Other'
         END AS PaymentDescription,
       SUM(taxi_trips.Total_Amount) AS Total_Amount
  FROM `taxi_dataset.taxi_trips` AS taxi_trips
 WHERE taxi_trips.Pickup_DateTime BETWEEN '2020-01-01' AND '2020-12-31' 
   AND Passenger_Count IS NOT NULL
   AND Payment_Type_Id IN (1,2,3,4)
 GROUP BY 1, 2, 3   
)
SELECT MonthName,
       CAST(Credit   AS INTEGER) / 1000 AS Credit,
       CAST(Cash     AS INTEGER) / 1000 AS Cash,
       CAST(Other    AS INTEGER) / 1000 AS Other
  FROM MonthlyData
 PIVOT(SUM(Total_Amount) FOR PaymentDescription IN ('Credit', 'Cash', 'Other'))
ORDER BY MonthNumber;

In [ ]:
display(myDataframe)

In [ ]:
ax = myDataframe.plot(kind="bar", x="MonthName", y=["Cash","Credit","Other"], figsize=(15, 7))
ax.set_title("Monthly Amounts by Payment Type")
ax.set_xlabel("Month")
ax.set_ylabel("Amount (thousands)")
ax.set_ylim([0, 100000])